## Extra, News, Strategies

In [1]:
%run strategies_functions.ipynb

## GUI

In [2]:
from PyQt5.QtWidgets import *
from PyQt5.QtCore import *
from PyQt5.QtGui import *
import pandas as pd
from datetime import datetime, timedelta
import numpy as np
import pandas as pd
import webbrowser
import urllib.request
from bs4 import BeautifulSoup
from PyQt5.QtCore import QAbstractTableModel, Qt, QVariant
from PyQt5 import QtWidgets
# Strategies
import tqdm
from tqdm import tnrange
import sys, time, csv, pprint, json, operator
from multiprocessing import Process
from scipy.spatial.distance import cdist
from scipy.spatial.distance import euclidean
import scipy.optimize as optimize
from cvxopt import solvers, matrix
solvers.options['show_progress'] = False
# Plotting
from matplotlib.backends.backend_qt5agg import FigureCanvasQTAgg as FigureCanvas
from matplotlib.figure import Figure
import matplotlib.pyplot as plt
import matplotlib.dates as dt
from mpl_toolkits.axes_grid1 import host_subplot
from scipy.interpolate import spline
from matplotlib.backends.backend_qt5agg import FigureCanvasQTAgg as FigureCanvas
from matplotlib.backends.backend_qt5agg import NavigationToolbar2QT as NavigationToolbar

app = QApplication([])
      
def basics_run(strategy, curr_pair, date0, date1, n_restarts):
    rfr = 2.3/100
    cumul_results = {}
    start_date = datetime.strptime(date0, '%Y-%m-%d')
    end_date = datetime.strptime(date1, '%Y-%m-%d') + timedelta(days=0, hours=23, minutes = 59, seconds=59)
    tc = 0.
    opts = {'display_interval': 1000}
    if curr_pair == 'EUR, GBP, JPY, CAD minutely':
        period = 'm'
    elif curr_pair == 'EUR, GBP, JPY, CAD hourly':
        period = 'h'
    elif curr_pair == 'EUR, GBP, JPY, CAD daily':
        period = 'd'
    
    if strategy == 'Sent_Analysis':    
        decay_txt = decayperiod.text()
        if decay_txt == '0':
            decay = 0
        elif decay_txt == '1':
            decay = 1
        elif decay_txt == '2':
            decay = 2
        elif decay_txt == '3':
            decay = 3
        elif decay_txt == '4':
            decay = 4
        elif decay_txt == '5':
            decay = 5
        elif decay_txt == '6':
            decay = 6
        elif decay_txt == '7':
            decay = 7
#     decay = 4
    
    # Prices
    df = pd.read_csv('Data/Testing_data/Prices/data_' + period + '_ret.csv')
    df['Gmt time'] = pd.to_datetime(df['Gmt time'])
    mask = (df['Gmt time'] > start_date) & (df['Gmt time'] <= end_date)
    df_fin = df.loc[mask]
    df_fin = df_fin.reset_index(drop=True)
    df_fin_np = np.array(df_fin.drop(['Gmt time'], axis=1))
    df_fin_np_time = np.array(df_fin)
    df_fin_date = np.array(df_fin[['Gmt time']])
    day_range = (end_date - start_date).days + 1
    if strategy == 'Anticor':
        results = anticor(df_fin_np, tc, opts)
    elif strategy == 'Anticor_Anticor':
        results = anticor_anticor(df_fin_np, tc, opts)
    elif strategy == 'PAMR':
        results = pamr(df_fin_np, tc, opts)
    elif strategy == 'RMR':
        results = rmr(df_fin_np, tc, opts)
    elif strategy == 'OLMAR':
        results = olmar(df_fin_np, tc, opts)
    elif strategy == 'ONS':
        results = ons(df_fin_np, tc, opts)
    elif strategy == 'EG':
        results = eg(df_fin_np, tc, opts)
    elif strategy == 'UP':
        results = up(df_fin_np, tc, opts)
    elif strategy == 'UCRP':
        results = ucrp(df_fin_np, tc, opts)
    elif strategy == 'Best Stock':
        results = best_stock(df_fin_np, tc, opts)
    elif strategy == 'BCRP':
        results = bcrp(df_fin_np, tc, opts)
    elif strategy == 'UBAH':
        results = ubah(df_fin_np, tc, opts)
    elif strategy == 'CORN':
        results = corn(df_fin_np, tc, opts)
    elif strategy == 'ARMA':
        results = arma(df_fin_np, tc, opts)
    elif strategy == 'ARIMA':
        results = arima(df_fin_np, tc, opts)
    elif strategy == 'CFR-OGD':
        results = cfr_ogd(df_fin_np, tc, opts)
    elif strategy == 'CFR-ONS':
        results = cfr_ons(df_fin_np, tc, opts)
    elif strategy == 'CFR-OGD-SA':
        results = cfr_ogd_sa(df_fin_np, df_fin_np_time, period, tc, opts)
    elif strategy == 'CFR-ONS-SA':
        results = cfr_ons_sa(df_fin_np, df_fin_np_time, period, tc, opts)
    elif strategy == 'Sent_Analysis':
        results = sentiment_strategy(date0, date1, period, decay)
        
    if strategy == 'Sent_Analysis':
        names_data = []
        cum_ret_data = []
        dates_intra_data = []
        dates_final_data = []
        daily_ret_intra_data = []
        daily_ret_final_data = []
        cumprod_ret_intra_data = []
        cumprod_ret_final_data = []
        for i, (strategy_name, res) in enumerate(results.items()):
            names_data.append(strategy_name)
            cum_ret_data.append(res[0])
            dates_intra_data.append(res[1])
            dates_final_data.append(res[2])
            daily_ret_intra_data.append(res[3])
            daily_ret_final_data.append(res[4])
            cumprod_ret_intra_data.append(res[5])
            cumprod_ret_final_data.append(res[6])
        cum_ret = 1 + cum_ret_data[0] + cum_ret_data[1] + cum_ret_data[2] + cum_ret_data[3]
        IR = 0.0
        Sharpe = 0.0
        MD = 0.0
        
    else:    
        for i, (strategy_name, res) in enumerate(results.items()):
            cum_ret = res[0]
            cumprod_ret = res[1]
            daily_ret = res[2]
            daily_portfolio = res[3]
        # Sharpe ratio 
        IR = np.mean(daily_ret) / np.std(daily_ret)
#         Sharpe = IR * np.sqrt(252) * 0.0001
#         MD = (np.max(cumprod_ret) - np.min(cumprod_ret)) / np.max(cumprod_ret)
    
    # Annualized return
    an_ret = (cum_ret**(252/day_range) - 1.0)*100.0
    Sharpe = round((float(np.array(an_ret))/100 - rfr) / np.std((cumprod_ret**(260/(day_range)) - 1.0) - rfr), 4)
    MD = round(((min(cumprod_ret) - max(cumprod_ret)) / max(cumprod_ret))*100, 4)
    
    table_res.horizontalHeaderItem(n_restarts).setText(strategy)
    table_res.setItem(0, n_restarts, QTableWidgetItem(curr_pair))
    table_res.setItem(1, n_restarts, QTableWidgetItem('{:.2f}%'.format(cum_ret*100 - 100)))
    table_res.setItem(2, n_restarts, QTableWidgetItem('{:.2f}%'.format(an_ret)))
    table_res.setItem(3, n_restarts, QTableWidgetItem('{:.2f}'.format(Sharpe)))
    table_res.setItem(4, n_restarts, QTableWidgetItem('{:.2f}%'.format(MD)))
    
    if strategy != 'Sent_Analysis':
        res_plot(cumprod_ret, df_fin_date, curr_pair, strategy)
        daily_plot(daily_ret, df_fin_date, curr_pair, strategy)
        exchr_dynamics_plot(df_fin_np, df_fin_date, curr_pair)
    elif strategy == 'Sent_Analysis':
        np_price_data = convert_prices(date0, date1, period, decay)
        np_sentiment_data_eur = convert_sentiment('EURUSD', date0, date1)
        np_sentiment_data_cad = convert_sentiment('CADUSD', date0, date1)
        np_sentiment_data_gbp = convert_sentiment('GBPUSD', date0, date1)
        np_sentiment_data_jpy = convert_sentiment('JPYUSD', date0, date1)
        
        sent_plot(np_sentiment_data_eur, np_price_data[2], 'EURUSD')
        sent_plot(np_sentiment_data_cad, np_price_data[3], 'CADUSD')
        sent_plot(np_sentiment_data_gbp, np_price_data[4], 'GBPUSD')
        sent_plot(np_sentiment_data_jpy, np_price_data[5], 'JPYUSD')
        
        exchr_dynamics_plot(df_fin_np, df_fin_date, curr_pair)
        daily_plot_sent(dates_intra_data, dates_final_data, daily_ret_intra_data, daily_ret_final_data)
        cumprod_plot_sent(dates_intra_data, dates_final_data, cumprod_ret_intra_data, cumprod_ret_final_data)
#         res_plot(cumprod_ret, df_fin_date, curr_pair, strategy)
#         daily_plot(daily_ret, df_fin_date, curr_pair, strategy)
    
def approach_change():
    if approach_2.isChecked():
        title3l.hide()
        start_date.hide()
        end_date.hide()
    elif approach_1.isChecked():
        title3l.show()
        start_date.show()
        end_date.show()
    
def graphs_change():
    if combo_strategy.currentText() == 'Sent_Analysis':
        tabs.addTab(graph4, 'EURUSD')
        tabs.addTab(graph5, 'CADUSD')
        tabs.addTab(graph6, 'GBPUSD')
        tabs.addTab(graph7, 'JPYUSD')
        graph4.show()
        graph5.show()
        graph6.show()
        graph7.show()
    elif combo_strategy.currentText() != 'Sent_Analysis':
        tabs.removeTab(6)
        tabs.removeTab(5)
        tabs.removeTab(4)
        tabs.removeTab(3)
        
def decay_change():
    if combo_strategy.currentText() == 'Sent_Analysis':
        title8l.show()
        combo_decay.show()
    else:
        title8l.hide()
        combo_decay.hide()

def OpenLink(item):
    if item.column() == 2:
        webbrowser.open(item.text(), new=2)

restarts = []
def run_analysis():
    stream_news(websiteName.text())   
    if tweet_button_1.isChecked():
        stream_tweets(currencyName.text())
    restarts.append('1')
    strategy_selection(currencyName.text(), 
                       start_date.date().toString('yyyy-MM-dd'),
                       end_date.date().toString('yyyy-MM-dd'),
                       len(restarts))
    
#################################### Choose Approach (real time vs. backtesting) ##############################
approach_1 = QRadioButton('Backtesting')
approach_2 = QRadioButton('Real time prediction')
approach_1.setChecked(True)
        
approaches_group = QButtonGroup()
approaches_group.addButton(approach_1)
approaches_group.addButton(approach_2)
approaches_group.buttonClicked.connect(approach_change)

layout_approaches = QHBoxLayout()
layout_approaches.addWidget(approach_1)
layout_approaches.addWidget(approach_2)
    
#################################### Strategies ##############################################################
strategyName = QLabel("")   
combo_strategy = QComboBox()
combo_strategy.addItems(['', 'CFR-OGD-SA', 'CFR-ONS-SA', 'CFR-OGD', 'CFR-ONS',
                        'Anticor', 'Anticor_Anticor', 'PAMR', 'RMR', 'OLMAR', 'ONS', 'EG', 'UP',
                        'UCRP', 'Best Stock', 'BCRP', 'UBAH', 'CORN', 'ARMA', 'ARIMA']) # 'Sent_Analysis',
# , 'CFR-OGD'])
combo_strategy.show()
combo_strategy.setStyleSheet('selection-background-color: rgb(168,168,168)')
combo_strategy.activated[str].connect(strategyName.setText)
combo_strategy.activated[str].connect(graphs_change)
combo_strategy.activated[str].connect(decay_change)

#################################### Choose currency ##########################################################
currencyName = QLabel("")   

combo = QComboBox()
combo.addItems(['', 'EUR, GBP, JPY, CAD minutely', 'EUR, GBP, JPY, CAD hourly', 'EUR, GBP, JPY, CAD daily'])
combo.show()
combo.setStyleSheet('selection-background-color: rgb(168,168,168)')
combo.activated[str].connect(currencyName.setText)

#################################### Choose decay ##########################################################
decayperiod = QLabel("")   

combo_decay = QComboBox()
combo_decay.addItems(['0', '1', '2', '3', '4', '5', '6', '7'])
for i in range(8):
    combo_decay.setItemData(i, 'Number of days from the end of investing period to close positions', role = Qt.ToolTipRole)
combo_decay.setStyleSheet('selection-background-color: rgb(168,168,168)')
combo_decay.activated[str].connect(decayperiod.setText)

#################################### Choose website for news streaming ########################################
websiteName = QLabel("")   
combo_news = QComboBox()
combo_news.addItems(['','БКС-аналитика', 'РосБизнесКонсалтинг (РБК)'])
# , 'Wall Street Journal (WSJ)', 'Financial Times (FT)'])
combo_news.show()
combo_news.setStyleSheet('selection-background-color: rgb(168,168,168)')
combo_news.activated[str].connect(websiteName.setText)

#################################### Choose dates ############################################################
start_date = QDateEdit()
start_date.setMinimumDate(QDate(2019, 1, 7))
# start_date.setMaximumDate(QDate.currentDate())
start_date.setMaximumDate(QDate(2019, 5, 25))
start_date.setDateTime(QDateTime.currentDateTime())
start_date.setCalendarPopup(True)

end_date = QDateEdit()
end_date.setDate(QDate.currentDate())
end_date.setMinimumDate(QDate(2019, 1, 7))
# end_date.setMaximumDate(QDate.currentDate())
end_date.setMaximumDate(QDate(2019, 5, 25))
end_date.setCalendarPopup(True)

layout_dates = QHBoxLayout()
layout_dates.addWidget(start_date)
layout_dates.addWidget(end_date)

#################################### STREAM TWEETS ##########################################################
tweet_button_1 = QRadioButton('Stream tweets')
tweet_button_2 = QRadioButton('NO tweets')
tweet_button_2.setChecked(True)

tweet_group = QButtonGroup()
tweet_group.addButton(tweet_button_1)
tweet_group.addButton(tweet_button_2)

layout_tweet = QHBoxLayout()
layout_tweet.addWidget(tweet_button_1)
layout_tweet.addWidget(tweet_button_2)

table_tw = QTableWidget()  
table_tw.setColumnCount(4)     
table_tw.setRowCount(10)
table_tw.setHorizontalHeaderLabels(["Author", "Tweet", "Classification", "Date"])
table_tw.horizontalHeaderItem(0).setTextAlignment(Qt.AlignCenter)
table_tw.horizontalHeaderItem(1).setTextAlignment(Qt.AlignCenter)
table_tw.horizontalHeaderItem(2).setTextAlignment(Qt.AlignCenter)
table_tw.setMinimumHeight(110)
# table_tw.setMaximumHeight(170)
font_t = QFont()
font_t.setPointSize(10)
table_tw.setFont(font_t)

################################ STREAM NEWS ###############################################################
button_start = QPushButton('START ANALYSIS')
button_start.setStyleSheet('QPushButton {background-color: rgb(173,255,47); font:bold;}')
button_start.show()
button_start.clicked.connect(lambda: run_analysis())

table_nw = QTableWidget()  
table_nw.setColumnCount(3)     
table_nw.setRowCount(10)
table_nw.setHorizontalHeaderLabels(["Heading", "Date and Time", "Url"])
table_nw.horizontalHeaderItem(0).setTextAlignment(Qt.AlignCenter)
table_nw.horizontalHeaderItem(1).setTextAlignment(Qt.AlignCenter)
table_nw.horizontalHeaderItem(2).setTextAlignment(Qt.AlignCenter)
table_nw.setColumnWidth(0, 250)
table_nw.setColumnWidth(1, 100)
table_nw.setColumnWidth(2, 80)
table_tw.setMinimumHeight(110)
# table_nw.setMaximumHeight(200)
font_t = QFont()
font_t.setPointSize(10)
table_nw.setFont(font_t)
table_nw.itemDoubleClicked.connect(OpenLink)

#################################### Show statistics ########################################################
exch_r = QLabel('{}'.format(currencyName.text()))

table_res = QTableWidget()   
table_res.setColumnCount(17)   # number of columns  
table_res.setRowCount(5)
table_res.setHorizontalHeaderLabels(["Metrics", "-", "-", "-", "-", "-", "-", "-", "-", "-", "-", "-", "-",
                                    "-", "-", "-", "-", "-"])
table_res.horizontalHeaderItem(0).setTextAlignment(Qt.AlignCenter)
table_res.setColumnWidth(0, 150)
for i in range(1, 17):
    table_res.setColumnWidth(i, 80)
    table_res.horizontalHeaderItem(i).setTextAlignment(Qt.AlignCenter)
font_res = QFont()
font_res.setPointSize(10)
table_res.setFont(font_res)
table_res.setItem(0, 0, QTableWidgetItem('Category of portfolio'))
table_res.setItem(1, 0, QTableWidgetItem('Return for the period'))
table_res.setItem(2, 0, QTableWidgetItem('Annualized return'))
table_res.setItem(3, 0, QTableWidgetItem('Sharpe Ratio'))
table_res.setItem(4, 0, QTableWidgetItem('Maximum Drawdown'))
table_res.setMinimumHeight(110)
# table_res.setMinimumHeight(110)

tabs = QTabWidget()
figure = plt.figure(figsize=(6,2))
graph = FigureCanvas(figure)
graph.setMinimumHeight(220)

figure2 = plt.figure(figsize=(6,2))
graph2 = FigureCanvas(figure2)
graph2.setMinimumHeight(220)

figure3 = plt.figure(figsize=(6,2))
graph3 = FigureCanvas(figure3)
graph3.setMinimumHeight(220)

figure4 = plt.figure(figsize=(6,2))
graph4 = FigureCanvas(figure4)
graph4.setMinimumHeight(220)

figure5 = plt.figure(figsize=(6,2))
graph5 = FigureCanvas(figure5)
graph5.setMinimumHeight(220)

figure6 = plt.figure(figsize=(6,2))
graph6 = FigureCanvas(figure6)
graph6.setMinimumHeight(220)

figure7 = plt.figure(figsize=(6,2))
graph7 = FigureCanvas(figure7)
graph7.setMinimumHeight(220)

tabs.addTab(graph, 'Cumulative portfolio return')
tabs.addTab(graph2, 'Daily portfolio return')
tabs.addTab(graph3, 'Exchange rate dynamics')
# toolbar = NavigationToolbar(graph, parent=None)

######################################### Progress Bar ######################################################
progress = QProgressBar()
progress.setMinimum(0)
progress.setMaximum(13)
progress.setValue(0)

#################################### General ################################################################
# --------------------------------- LEFT HALF OF THE GUI ---------------------------------------------------
# Approach
title1l = QLabel("Choose approach")
title1l.setAlignment(Qt.AlignCenter)
title1l.setStyleSheet("font: bold")
layout1l = QVBoxLayout()
layout1l.addWidget(title1l)
layout1l.addLayout(layout_approaches)
frame1l = QFrame()
frame1l.setFrameShape(QFrame.StyledPanel)
frame1l.setLineWidth(1)
frame1l.setStyleSheet("background-color:rgb(247,246,246)")
frame1l.setLayout(layout1l)

# Currency
title2l = QLabel("Choose portfolio category")      
title2l.setAlignment(Qt.AlignCenter) 
title2l.setStyleSheet("font: bold")
layout2l = QVBoxLayout()
layout2l.addWidget(title2l)
layout2l.addWidget(combo)
frame2l = QFrame()
frame2l.setFrameShape(QFrame.StyledPanel)
frame2l.setStyleSheet("background-color:rgb(247,246,246)")
frame2l.setLayout(layout2l)

# Dates
title3l = QLabel("Choose dates for analysis")      
title3l.setAlignment(Qt.AlignCenter)
title3l.setStyleSheet("font: bold")
layout3l = QVBoxLayout()
layout3l.addWidget(title3l)
layout3l.addLayout(layout_dates)
frame3l = QFrame()
frame3l.setFrameShape(QFrame.StyledPanel)
frame3l.setLineWidth(1)
frame3l.setStyleSheet("background-color:rgb(247,246,246)")
frame3l.setLayout(layout3l)

# Strategies
title4l = QLabel("Choose strategy")      
title4l.setAlignment(Qt.AlignCenter) 
title4l.setStyleSheet("font: bold")
layout4l = QVBoxLayout()
layout4l.addWidget(title4l)
layout4l.addWidget(combo_strategy)
frame4l = QFrame()
frame4l.setFrameShape(QFrame.StyledPanel)
frame4l.setLineWidth(1)
frame4l.setStyleSheet("background-color:rgb(247,246,246)")
frame4l.setLayout(layout4l)

# Decay
title8l = QLabel("Choose decay")      
title8l.setAlignment(Qt.AlignCenter) 
title8l.setStyleSheet("font: bold")
layout8l = QVBoxLayout()
layout8l.addWidget(title8l)
layout8l.addWidget(combo_decay)
frame8l = QFrame()
frame8l.setFrameShape(QFrame.StyledPanel)
frame8l.setLineWidth(1)
frame8l.setStyleSheet("background-color:rgb(247,246,246)")
frame8l.setLayout(layout8l)

# Stream news - selection
title5l = QLabel("Choose website for news streaming:") 
title5l.setAlignment(Qt.AlignCenter) 
title5l.setStyleSheet("font: bold")
layout5l = QVBoxLayout()
layout5l.addWidget(title5l)
layout5l.addWidget(combo_news)
frame5l = QFrame()
frame5l.setFrameShape(QFrame.StyledPanel)
frame5l.setLineWidth(1)
frame5l.setStyleSheet("background-color:rgb(247,246,246)")
frame5l.setLayout(layout5l)

# Stream tweets - selection
title6l = QLabel("Stream tweets option")      
title6l.setAlignment(Qt.AlignCenter) 
title6l.setStyleSheet("font: bold")
layout6l = QVBoxLayout()
layout6l.addWidget(title6l)
layout6l.addLayout(layout_tweet)
frame6l = QFrame()
frame6l.setFrameShape(QFrame.StyledPanel)
frame6l.setLineWidth(1)
frame6l.setStyleSheet("background-color:rgb(247,246,246)")
frame6l.setLayout(layout6l)

# Button to run all analysis
layout7l = QVBoxLayout()
layout7l.addWidget(button_start)
frame7l = QFrame()
frame7l.setFrameShape(QFrame.StyledPanel)
frame7l.setStyleSheet("background-color:rgb(247,246,246)")
frame7l.setLayout(layout7l)

layout_left = QVBoxLayout()
layout_left.addWidget(frame1l)
layout_left.addWidget(frame2l)
layout_left.addWidget(frame3l)
layout_left.addWidget(frame4l)
layout_left.addWidget(frame8l)
layout_left.addWidget(frame5l)
layout_left.addWidget(frame6l)
layout_left.addWidget(frame7l)

widget_left = QWidget()
widget_left.setLayout(layout_left)

# --------------------------------- RIGHT HALF OF THE GUI ---------------------------------------------
#___________________________________________ TOP _______________________________________________________
# Display of news
title1r = QLabel("Top currency market news") 
title1r.setAlignment(Qt.AlignCenter) 
title1r.setStyleSheet("font: bold")
layout1r = QVBoxLayout()
layout1r.setContentsMargins(0,0,0,0)
layout1r.addWidget(title1r)
layout1r.addWidget(table_nw)
widget1r = QWidget()
widget1r.setLayout(layout1r)

# Display of tweets
title3r = QLabel('Top tweets')
title3r.setAlignment(Qt.AlignCenter) 
title3r.setStyleSheet("font: bold")
layout3r = QVBoxLayout()
layout3r.addWidget(title3r)
layout3r.addWidget(table_tw)
layout3r.setContentsMargins(0,0,0,0)
widget3r = QWidget()
widget3r.setLayout(layout3r)

# Display news and tweets on the same level
layout13r = QGridLayout()
layout13r.addWidget(widget1r, 0, 0)
layout13r.addWidget(widget3r, 0, 1)
layout13r.setContentsMargins(0,0,0,0)
widget13r = QWidget()
widget13r.setLayout(layout13r)

# Display graph
title4r = QLabel("Graphs of exchange rates, returns and sentiment") 
title4r.setAlignment(Qt.AlignCenter) 
title4r.setStyleSheet("font: bold")
layout4r = QVBoxLayout()
layout4r.addWidget(title4r)
layout4r.addWidget(tabs)
layout4r.setContentsMargins(0,0,0,0)
widget4r = QWidget()
widget4r.setLayout(layout4r)

# Display results
title5r = QLabel("Quantitative results of strategies implementation") 
title5r.setAlignment(Qt.AlignCenter) 
title5r.setStyleSheet("font: bold")
layout5r = QVBoxLayout()
layout5r.addWidget(title5r)
layout5r.addWidget(table_res)
layout5r.setContentsMargins(0,0,0,0)
# layout5r.addWidget(progress)
widget5r = QWidget()
widget5r.setLayout(layout5r)

layout_right = QVBoxLayout()
layout_right.addWidget(widget13r)
layout_right.addWidget(widget4r)
layout_right.addWidget(widget5r)
widget_right = QWidget()
widget_right.setLayout(layout_right)

splitter = QSplitter(Qt.Horizontal)
splitter.addWidget(widget_left)
splitter.addWidget(widget_right)
splitter.setStretchFactor(1, 1)

layout_general = QHBoxLayout()
layout_general.addWidget(splitter)

window = QWidget()
window.setLayout(layout_general)
window.setWindowState(Qt.WindowMaximized)
window.show()

app.exec_()

100%|██████████| 71/71 [00:00<00:00, 17342.94it/s]
/anaconda3/lib/python3.6/site-packages/matplotlib/figure.py:98: MatplotlibDeprecationWarning: 
Adding an axes using the same arguments as a previous axes currently reuses the earlier instance.  In a future version, a new instance will always be created and returned.  Meanwhile, this warning can be suppressed, and the future behavior ensured, by passing a unique label to each axes instance.
  "Adding an axes using the same arguments as a previous axes "


/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:386: RuntimeWarning: Mean of empty slice.
/anaconda3/lib/python3.6/site-packages/numpy/core/_methods.py:85: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


[[0.99979258 0.99981677 0.99944813 0.99940999 0.99944851]
 [1.00014699 1.00019609 0.99973211 0.99968276 0.9997326 ]
 [1.00128057 1.00137511 1.00135948 1.00138531 1.00135922]
 [1.0021841  1.0021916  1.00226682 1.00229478 1.00226654]]
[[0.99993448 0.99986659 0.99994665 0.99995167 0.9999466 ]
 [1.0000377  1.00010399 1.0000072  0.99998507 1.00000743]
 [1.00171322 1.00160083 1.00173099 1.0017733  1.00173056]
 [1.00214745 1.00218063 1.0021387  1.00215019 1.00213859]]
[[0.99995579 1.00001796 1.00061502 1.00066993 1.00061447]
 [1.00001628 1.00001783 0.99965052 0.99961272 0.9996509 ]
 [1.00172909 1.00167501 1.00111942 1.00110676 1.00111954]
 [1.0021481  1.00222632 1.00233982 1.00235665 1.00233965]]
[[0.99996566 1.00006071 1.0004812  1.00051803 1.00048082]
 [1.00002326 1.00006536 1.0005797  1.00061352 1.00057936]
 [1.00168796 1.0015636  1.00129729 1.00129092 1.00129736]
 [1.00215693 1.00219525 1.00189161 1.00186989 1.00189183]]
[[1.00021224 1.00049045 1.00196315 1.00209486 1.00196182]
 [1.000074

[[1.00292618 1.00285077 1.0031517  1.0031654  1.00315156]
 [0.99813305 0.99806584 0.99796438 0.99792053 0.99796482]
 [1.00964132 1.00917837 1.00972715 1.00979819 1.00972643]
 [1.00120546 1.00096145 1.00154516 1.00160659 1.00154454]]
[[1.00292397 1.00288299 1.00281138 1.00281492 1.00281135]
 [0.9981269  0.99804504 0.99808363 0.99808054 0.99808367]
 [1.00965919 1.00951745 1.0096721  1.00974278 1.00967139]
 [1.00127696 1.0012338  1.00152664 1.001565   1.00152626]]
[[1.00290696 1.00290625 1.00267702 1.00266816 1.00267711]
 [0.99815793 0.998152   0.99823237 0.99824708 0.99823222]
 [1.00965804 1.00964153 1.00939661 1.00942881 1.00939629]
 [1.00131499 1.00135809 1.00125217 1.00126111 1.00125208]]
[[1.00286466 1.00284539 1.00261434 1.00260008 1.00261448]
 [0.99820616 0.99821439 0.99844044 0.99846455 0.99844019]
 [1.00961533 1.00957865 1.00935705 1.0093624  1.009357  ]
 [1.00131126 1.00121187 1.00047482 1.00043266 1.00047524]]
[[1.00273342 1.00277917 1.0026412  1.00262916 1.00264132]
 [0.998211

0